In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'd:\\study\\own study\\OutSourceProjects\\Polio ChatBot\\traning'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [5]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
# extracted_data

In [6]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [19]:
# %pip install -U sentence-transformers

In [20]:

from langchain.embeddings import HuggingFaceEmbeddings

In [21]:
#Download the Embeddings from Hugging Face
# The model link = https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
# 384 dimensional dense vector space

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [22]:
embeddings = download_hugging_face_embeddings()

In [23]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [24]:
from dotenv import load_dotenv
load_dotenv()

False

In [33]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
PINECONE_API_KEY = "pcsk_6WAvF3_4LqaUuVrohRhXVHhPxDnX7M82ehi6jqaeMHtgTGCAw3fmobvfNUAtHwh5b5ebQM"
print(PINECONE_API_KEY)
# OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

pcsk_6WAvF3_4LqaUuVrohRhXVHhPxDnX7M82ehi6jqaeMHtgTGCAw3fmobvfNUAtHwh5b5ebQM


In [34]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [35]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [36]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [37]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [38]:
docsearch

In [39]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [58]:
querry = "Tell me which medicne we user for it"
retrieved_docs = retriever.invoke(querry)

In [47]:
retrieved_docs

[Document(id='d2725cc0-d5a2-4853-ad21-0241f4e256b8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a8d89f84-5e67-4f2a-a161-8d9f58caaab4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteri

In [43]:
from groq import Groq
groq_client = Groq(api_key="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")

In [59]:
prompt = f"""###INSTRUCTIONS###
            "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
            ###CONTEXT###
            {retrieved_docs}

            ###QUERY###
            {querry}
        """

response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{
            "role": "system",
            "content": "You are a Medical information specialist. First determine question relevance. Never answer non-medical questions."
        },{
            "role": "user", 
            "content": prompt
        }],
        temperature=0.3,  # Lower temp for more consistent structure
        max_tokens=500
    )

full_response = response.choices[0].message.content.strip()
    

In [60]:
full_response

"I don't know which medicine you are referring to, as the question is incomplete. The provided context appears to be from a medical encyclopedia, but it does not specify a particular medicine or condition. I would need more information to provide a relevant answer."

In [42]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])